In [1]:
! pip install pandas
! pip install petl
! pip install glob


You should consider upgrading via the 'C:\Users\adamc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\adamc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob
You should consider upgrading via the 'C:\Users\adamc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import petl as etl
import os
import glob

1. Creating dataframe of all CSV files

In [3]:
# CSV FILES APPENDING TO A SINGLE DATAFRAME

path = "branch_data/sales_by_branch"
all_csv_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_csv_file = (pd
                    .read_csv(file, sep=',')
                    .rename(columns={
                        'total_quantity': 'quantity', 
                        'quantity_purchased': 'quantity',
                        'total_quantity_purchased': 'quantity',
                        'item': 'product',
                        'sku': 'product',
                        })
                    .assign(branch = file[38:][:-4].replace('_', ' ')) 
                    for file in all_csv_files)


df_csv_merged = pd.concat(df_from_each_csv_file, ignore_index=True)

# df_csv_merged = pd.DataFrame(df_csv_merged)
df_csv_merged.head()

C:\Users\adamc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\reshape\concat.py:348: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)


,year,month,day,hour,product,quantity,amount_in_gbp,branch
0,2010,1,9,0,chocolate twist,182,145.6,Bassetlaw outlet
1,2010,1,9,0,plastic sunshades,98,313.6,Bassetlaw outlet
2,2010,1,9,0,lg washing machine,151,12080.0,Bassetlaw outlet
3,2010,1,9,0,fresh whole chicken,230,257.6,Bassetlaw outlet
4,2010,1,9,0,free range large eggs x12,150,180.0,Bassetlaw outlet


In [4]:
# renaming dataframe to be reused for both CSV and JSON dataframes
sales_df = df_csv_merged

2. Formatting of main dataframes

In [5]:
'''1 Convert to table'''
sales_df = etl.fromdataframe(sales_df)
print(sales_df)

+------+-------+-----+------+---------------------------+----------+--------------------+------------------+
| year | month | day | hour | product                   | quantity | amount_in_gbp      | branch           |
+======+=======+=====+======+===========================+==========+====================+==================+
| 2010 |     1 |   9 |    0 | chocolate twist           | 182      |              145.6 | Bassetlaw outlet |
+------+-------+-----+------+---------------------------+----------+--------------------+------------------+
| 2010 |     1 |   9 |    0 | plastic sunshades         | 98       |              313.6 | Bassetlaw outlet |
+------+-------+-----+------+---------------------------+----------+--------------------+------------------+
| 2010 |     1 |   9 |    0 | lg washing machine        | 151      |            12080.0 | Bassetlaw outlet |
+------+-------+-----+------+---------------------------+----------+--------------------+------------------+
| 2010 |     1 |   

In [6]:
'''2 replace '-' with '0' '''
sales_df = etl.replaceall(sales_df, '-', '0')
print(sales_df)

+------+-------+-----+------+---------------------------+----------+--------------------+------------------+
| year | month | day | hour | product                   | quantity | amount_in_gbp      | branch           |
+======+=======+=====+======+===========================+==========+====================+==================+
| 2010 |     1 |   9 |    0 | chocolate twist           | 182      |              145.6 | Bassetlaw outlet |
+------+-------+-----+------+---------------------------+----------+--------------------+------------------+
| 2010 |     1 |   9 |    0 | plastic sunshades         | 98       |              313.6 | Bassetlaw outlet |
+------+-------+-----+------+---------------------------+----------+--------------------+------------------+
| 2010 |     1 |   9 |    0 | lg washing machine        | 151      |            12080.0 | Bassetlaw outlet |
+------+-------+-----+------+---------------------------+----------+--------------------+------------------+
| 2010 |     1 |   

In [7]:
'''4 convert to numbers'''
sales_df = etl.addfield(sales_df, 'qty', lambda row: int(row.quantity))
sales_df = etl.cutout(sales_df, 'quantity')
sales_df



year,month,day,hour,product,amount_in_gbp,branch,qty
2010,1,9,0,chocolate twist,145.6,Bassetlaw outlet,182
2010,1,9,0,plastic sunshades,313.6,Bassetlaw outlet,98
2010,1,9,0,lg washing machine,12080.0,Bassetlaw outlet,151
2010,1,9,0,fresh whole chicken,257.6,Bassetlaw outlet,230
2010,1,9,0,free range large eggs x12,180.00000000000003,Bassetlaw outlet,150


3. Save table to CSV file
    - this is quicker than converting to dataframe

In [8]:
etl.tocsv(sales_df,'data_output/1_csv_files_dataframe.csv')